## Prerequisites

Before delving into the fine-tuning process, ensure that you have the following prerequisites in place:

1. **GPU**: This tutorial cannot run on free Google Colab; it requires more powerful GPUs, such as the A100.
2. **Python Packages**: Ensure that you have the necessary Python packages installed. You can use the following commands to install them:

Let's begin by checking if your GPU is correctly detected:

Let's define a wrapper function which will get completion from the model from a user question

Let's define a wrapper function which will get completion from the model from a user question

## Step 1 Set up

### Step 1.1 - Set Credentials

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
from huggingface_hub import login
login()

In [ ]:
# Setup Kaggle API Credentials:
# Make sure kaggle.json file exists in this folder
!mkdir ~/.kaggle
!cp '/content/drive/My Drive/Colab Notebooks/kaggle.json' ~/.kaggle/
!ls ~/.kaggle/

# Change the permissions of the file.
!chmod 600 ~/.kaggle/kaggle.json

### Step 1.2 - Set variables

In [10]:
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/model'
MODEL_NAME = 'llm-prompt-recovery'

HUGGING_FACE_USERNAME = 'valentinalilova'
# print(f'{HUGGING_FACE_USERNAME}/{MODEL_NAME}')

TRAIN_FILE_PATH = '/content/drive/My Drive/Colab Notebooks/train_example.csv'

# Set the name for the new dataset and a path for the files
KAGGLE_DATASET_PATH = '/content/dataset'
KAGGLE_DATASET_NAME = MODEL_NAME
KAGGLE_USERNAME = HUGGING_FACE_USERNAME

### Step 1.3 - Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [5]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

### 1.4 Imports

In [ ]:
import pandas as pd
import os
import torch

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
 )


model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
from peft import PeftConfig, PeftModel
adapter_model_name = "dfurman/Mistral-7B-Instruct-v0.2"
model = PeftModel.from_pretrained(model, adapter_model_name)

adapter_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Now we specify the model ID and then we load it with our previously defined quantization configuration.

Run a inference on the base model. The model does not seem to understand our instruction and gives us a list of questions related to our query.

## Step 3 - Load dataset for finetuning

### Lets Load the Dataset

For this tutorial, we will fine-tune Mistral 7B Instruct for code generation.

We will be using this [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) which is curated by [TokenBender (e/xperiments)](https://twitter.com/4evaBehindSOTA) and is an excellent data source for fine-tuning models for code generation. It follows the alpaca style of instructions, which is an excellent starting point for this task. The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [8]:
df = pd.read_csv(TRAIN_FILE_PATH)

def create_prompt(original, rewritten):
    return f"Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.\nYou are trying to understand how the original essay was transformed into a new version.\nAnalyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.\nOnly give me the PROMPT. Start directly with the prompt, that's all I need.\nOutput should be only line ONLY\nOriginal Essay: {original}\nRewritten Essay: {rewritten}"

# Apply the function to each row to create the prompt column
df['prompt'] = df.apply(lambda row: create_prompt(row['original_text'], row['rewritten_text']), axis=1)

In [9]:
df.head(10)

,,,,,,,,,,,,,,,id,original_text,rewrite_prompt,rewritten_text,prompt
-1,"The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text. Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.","Convert this into a sea shanty: ""The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text. Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."",Here is your shanty: (Verse 1) The text is rewritten",the LLM has spun,With prompts so clever,they've been outrun. The goal is to find,the prompt so bright,To crack the code,and shine the light. (Chorus) Oh,this is a code competition,my dear,With text and prompts,we'll compete. Two thousand texts,a challenge grand,To guess the prompts,hand over hand.(Verse 2) The original text,a treasure lost,The rewrite prompt,"a secret to be""","Given are 2 essays, the Rewritten essay was cr..."
0,"The process of photosynthesis is fundamental to life on Earth, as it is through this process that plants convert light energy into chemical energy, thereby producing oxygen as a byproduct.","Summarize this in a tweet: ""The process of photosynthesis is fundamental to life on Earth, as it is through this process that plants convert light energy into chemical energy, thereby producing oxygen as a byproduct."",Photosynthesis: Earth's lifeline! Plants turning light into life-giving energy and oxygen. #NatureMagic #Science""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Given are 2 essays, the Rewritten essay was cr..."
1,Quantum computing is a rapidly growing field that promises to revolutionize computing by leveraging the principles of quantum mechanics to process information in ways that traditional computers cannot.,"Explain like I'm 5: ""Quantum computing is a rapidly growing field that promises to revolutionize computing by leveraging the principles of quantum mechanics to process information in ways that traditional computers cannot."",""So, imagine if your toys could do things by themselves by playing a special type of hide and seek with each other. That's kinda like quantum computing!""\n2,The French Revolution",which began in 1789,"was a pivotal period in French history that led to the rise of the French Republic and the end of monarchical rule.""","Write this as a historical diary entry: ""The French Revolution, which began in 1789, was a pivotal period in French history that led to the rise of the French Republic and the end of monarchical rule."",""Dear Diary, today marks the beginning of an upheaval. The year 1789 has brought the storm of change to France, ending the reign of kings.""\n3,Machine learning algorithms have transformed the tech industry by enabling the development of applications that can make predictions or decisions without being explicitly programmed to perform the task.""","Create a marketing slogan: ""Machine learning algorithms have transformed the tech industry by enabling the development of applications that can make predictions or decisions without being explicitly programmed to perform the task."",""Revolutionizing tech with a brain of its own! Machine learning - predicting the future, today. #TechRevolution""\n4,Blockchain technology underpins cryptocurrencies like Bitcoin","offering a decentralized ledger for secure transactions.""","Describe as a science fiction scenario: ""Blockchain technology underpins cryptocurrencies like Bitcoin, offering a de

Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

### Formatting the Dataset

Now, let's format the dataset in the required [Mistral-7B-Instruct-v0.1 format](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1).

> Many tutorials and blogs skip over this part, but I feel this is a really important step.

We'll put each instruction and input pair between `[INST]` and `[/INST]` output after that, like this:

```
<s>[INST] What is your favorite condiment? [/INST]
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavor to whatever I'm cooking up in the kitchen!</s>
```

You can use the following code to process your dataset and create a JSONL file in the correct format:

In [10]:
def generate_prompt(row):
    """Generate input text based on a prompt, task instruction, and answer.

    :param row: Series: Data point (row of a pandas DataFrame)
    :return: str: generated prompt text
    """
    # Assuming 'prompt' is the generated context and 'rewrite_prompt' is the instruction
    return f"<s>[INST]{row['prompt']} [/INST]{row['rewrite_prompt']}</s>"

# Apply the function to each row to transform the 'prompt' and 'rewrite_prompt' columns
df['formatted_prompt'] = df.apply(generate_prompt, axis=1)

In [11]:
from datasets import Dataset
df = df[['prompt', 'rewrite_prompt']]

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.select(range(10)) # ToDo: remove that
# Shuffle the dataset with a seed for reproducibility
dataset = dataset.shuffle(seed=1234)

# Tokenize the prompts in the dataset.
# This example assumes that the model requires only 'input_ids'.
# Adjust as necessary for your model, e.g., adding 'attention_mask'.
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

# Split the dataset into training and testing sets (90% training, 10% testing)
train_test_split = dataset.train_test_split(test_size=0.1)
train_data = train_test_split['train']
test_data = train_test_split['test']

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

We'll need to tokenize our data so the model can understand.


Split dataset into 90% for training and 10% for testing

### After Formatting, We should get something like this

```json
{
"text":"<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]
# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>",
"instruction":"Create a function to calculate the sum of a sequence of integers",
"input":"[1, 2, 3, 4, 5]",
"output":"# Python code def sum_sequence(sequence): sum = 0 for num in,
 sequence: sum += num return sum"
"prompt":"<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]
# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>"

}
```

While using SFT (**[Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/main/en/sft_trainer)**) for fine-tuning, we will be only passing in the “text” column of the dataset for fine-tuning.

In [12]:
print(test_data)

Dataset({
    features: ['prompt', 'rewrite_prompt', '__index_level_0__', '__index_level_1__', '__index_level_2__', '__index_level_3__', '__index_level_4__', '__index_level_5__', '__index_level_6__', '__index_level_7__', '__index_level_8__', '__index_level_9__', '__index_level_10__', '__index_level_11__', '__index_level_12__', '__index_level_13__', '__index_level_14__', 'input_ids', 'attention_mask'],
    num_rows: 1
})


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [13]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

Use the following function to find out the linear layers for fine tuning.
QLoRA paper : "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

In [15]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [16]:
modules = find_all_linear_names(model)
print(modules)

['up_proj', 'base_layer', 'down_proj', 'gate_proj']


In [17]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [18]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


Trainable: 20971520 | total: 7317229568 | Percentage: 0.2866%


## Step 5 - Run the training!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [19]:
# from datasets import load_dataset
# data = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split='train')
# data = data.train_test_split(test_size=0.1)
# train_data = data["train"]
# test_data = data["test"]

In [20]:
# import transformers
# tokenizer.pad_token = tokenizer.eos_token

# trainer = transformers.Trainer(
#      model=model,
#      train_dataset=train_data,
#      eval_dataset=test_data,
#      args=transformers.TrainingArguments(
#          per_device_train_batch_size=1,
#          gradient_accumulation_steps=4,
#          warmup_ratio=0.03,
#          max_steps=100,
#          learning_rate=2e-4,
#          fp16=True,
#          logging_steps=1,
#          output_dir="outputs_mistral_b_finance_finetuned_test",
#          optim="paged_adamw_8bit",
#          save_strategy="epoch",
#      ),
#      data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
#  )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [21]:
#new code using SFTTrainer
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Start the training

### Let's start the training process

In [22]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()


Step,Training Loss
1,3.430700
2,3.464000
3,2.081900
4,1.352900
5,0.634800
6,0.172800
7,0.363000
8,0.166200
9,0.100600
10,0.085600


TrainOutput(global_step=100, training_loss=0.12533549451152795, metrics={'train_runtime': 778.9605, 'train_samples_per_second': 0.514, 'train_steps_per_second': 0.128, 'total_flos': 2153777657020416.0, 'train_loss': 0.12533549451152795, 'epoch': 44.44})

## Save the model

### Save to Hugging Face

In [25]:
# # Create repo if needed
# from huggingface_hub import create_repo
# create_repo(MODEL_NAME, private=True)

In [31]:
# Clone the repo
from huggingface_hub import Repository
repo = Repository(MODEL_PATH, clone_from=f'{HUGGING_FACE_USERNAME}/{MODEL_NAME}')

Cloning https://huggingface.co/valentinalilova/llm-prompt-recovery into local empty directory.


In [47]:
# Save the model to the repo
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

('/content/drive/My Drive/Colab Notebooks/model/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/model/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/model/tokenizer.model',
 '/content/drive/My Drive/Colab Notebooks/model/added_tokens.json',
 '/content/drive/My Drive/Colab Notebooks/model/tokenizer.json')

In [61]:
# # Debugging
# %cd $MODEL_PATH
# !git status
# # %cd /content

/content/drive/My Drive/Colab Notebooks/model
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [48]:
# # Debugging
# !cat .git/hooks/pre-push

In [58]:
# # Fix push loading forever
# !chmod +x .git/hooks/pre-push
# !git lfs install

In [ ]:
repo.push_to_hub(commit_message="Upload model")
# # Other ways to push
# repo.git_push()
# !git push --verbose

### Save to Kaggle


In [24]:
# Create a New Kaggle Dataset:
# Create a directory for the Kaggle dataset
os.makedirs(KAGGLE_DATASET_PATH, exist_ok=True)

# Move the model files to this directory
!cp "$MODEL_PATH"/* "$KAGGLE_DATASET_PATH"/
!ls $KAGGLE_DATASET_PATH

# Create metadata file
metadata = {
    'title': KAGGLE_DATASET_NAME,
    'id': f'{KAGGLE_USERNAME}/{KAGGLE_DATASET_NAME}',
    'licenses': [{'name': 'CC0-1.0'}] # ToDo: Use that license? What metadata?
}

adapter_config.json	   dataset-metadata.json  special_tokens_map.json  tokenizer.json
adapter_model.safetensors  README.md		  tokenizer_config.json    tokenizer.model


In [26]:
import json
with open(os.path.join(KAGGLE_DATASET_PATH, "dataset-metadata.json"), "w") as f:
    json.dump(metadata, f)

In [27]:
# Upload the dataset to Kaggle
!kaggle datasets create -p {KAGGLE_DATASET_PATH} --dir-mode zip

Starting upload for file tokenizer.json
100% 1.71M/1.71M [00:00<00:00, 4.14MB/s]
Upload successful: tokenizer.json (2MB)
Starting upload for file tokenizer.model
100% 482k/482k [00:00<00:00, 1.21MB/s]
Upload successful: tokenizer.model (482KB)
Starting upload for file README.md
100% 5.00k/5.00k [00:00<00:00, 12.4kB/s]
Upload successful: README.md (5KB)
Starting upload for file tokenizer_config.json
100% 1.43k/1.43k [00:00<00:00, 3.46kB/s]
Upload successful: tokenizer_config.json (1KB)
Starting upload for file special_tokens_map.json
100% 437/437 [00:00<00:00, 1.11kB/s]
Upload successful: special_tokens_map.json (437B)
Starting upload for file adapter_model.safetensors
100% 288M/288M [00:03<00:00, 81.8MB/s]
Upload successful: adapter_model.safetensors (288MB)
Starting upload for file adapter_config.json
100% 666/666 [00:00<00:00, 1.43kB/s]
Upload successful: adapter_config.json (666B)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/valenti

## Step 6 Evaluating the model qualitatively: run an inference!



In [62]:
# import re
# def remove_special_characters(text):
#     # This regex will match any character that is not a letter, number, or whitespace
#     pattern = r'[^a-zA-Z0-9\s]'
#     text =text.replace("Transform" ,"improve")
#     text =text.replace("Reimagine" ,"rewrite")
#     # Replace these characters with an empty string
#     clean_text = re.sub(pattern, '', text)
#     return clean_text

# def get_completion(prompt) -> str:
#     device = "cuda:0"
#     print('prompt')
#     print(prompt)
#     encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

#     model_inputs = encodeds.to(device)


#     generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
#     generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print('gen')
#     print(generated_text)
#     prompt_length = len(tokenizer.encode(prompt))
#     print('output')
#     prompt_tokens = tokenizer.encode(prompt, add_special_tokens=False)
#     generated_tokens = generated_ids[0][len(prompt_tokens):]
#     generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
#     print(generated_text)
#     return generated_text

In [63]:
# test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
# test_df['prompt'] = test_df.apply(lambda row: create_prompt(row['original_text'], row['rewritten_text']), axis=1)
# test_df['prompt'] = test_df.apply(generate_prompt, axis=1)
# test_df['rewrite_prompt'] = test_df.apply(lambda row: get_completion(row['prompt']), axis=1)
# test_df = test_df[['id', 'rewrite_prompt']]
# test_df

In [64]:
# test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
# adject =[]
# for index, row in test_df.iterrows():
#         result = generate_prompt(create_prompt(row['original_text'], row['rewritten_text']))
#         result  = remove_special_characters(result)
#         print(result)
#         test_df.at[index, 'rewrite_prompt'] =  result




# test_df = test_df[['id', 'rewrite_prompt']]
# test_df


In [65]:
# test_df.to_csv('submission.csv', index=False)